In [1]:

%reload_ext autoreload
%autoreload 2
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

In [2]:
import numpy as np
from collections import OrderedDict

from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

design = designs.DesignPlanar()
gui = MetalGUI(design)

design.overwrite_enabled = True

In [3]:
design.variables['cpw_width'] = '10 um'
design.variables['cpw_gap'] = '6 um'
design._chips['main']['size']['size_x'] = '9mm'
design._chips['main']['size']['size_y'] = '6.5mm'
transmon_options = dict(
    connection_pads=dict(
        a = dict(loc_W=+1, loc_H=-1, pad_width='70um', cpw_extend = '50um'),
        b = dict(loc_W=-1, loc_H=-1, pad_width='125um', cpw_extend = '50um'),
        c = dict(loc_W=-1, loc_H=+1, pad_width='110um', cpw_extend = '50um'),
        d = dict(loc_W=+1, loc_H=+1, pad_width='110um', cpw_extend = '50um')
    ),
    gds_cell_name='FakeJunction_01',
    cl_off_center = '-50um',
    cl_pocket_edge = '180'
)



In [4]:

q1 = TransmonPocketCL(design, 'Q1', options = dict(
    pos_x='+1.1mm', pos_y='+0.8mm', orientation = '0', **transmon_options))
q2 = TransmonPocketCL(design, 'Q2', options = dict(
    pos_x='+2.7mm', pos_y='+0.8mm', orientation = '0', **transmon_options))
q3 = TransmonPocketCL(design, 'Q3', options = dict(
    pos_x='+2.7mm', pos_y='-0.3mm', orientation = '0', **transmon_options))
q4 = TransmonPocketCL(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='-0.7mm', orientation = '0', **transmon_options))
q5 = TransmonPocketCL(design, 'Q5', options = dict(
    pos_x='-2.7mm', pos_y='-0.3mm', orientation = '0', **transmon_options))
q6 = TransmonPocketCL(design, 'Q6', options = dict(
    pos_x='-1.1mm', pos_y='+0.8mm', orientation = '0', **transmon_options))
q7 = TransmonPocketCL(design, 'Q7', options = dict(
    pos_x='-2.7mm', pos_y='+0.8mm', orientation = '0', **transmon_options))


gui.rebuild()
gui.autoscale()

In [5]:
offset_tm = 69  

fillet='99.99um'
cpw_options = Dict(
    lead=Dict(
        start_straight='100um',
        end_straight='250um'),
    fillet=fillet
    )

def connect(cpw_name: str, pin1_comp_name: str, pin1_comp_pin: str, pin2_comp_name: str, pin2_comp_pin: str,
            length: str, asymmetry='0 um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        pin_inputs=Dict(
            start_pin=Dict(
                component=pin1_comp_name,
                pin=pin1_comp_pin),
            end_pin=Dict(
                component=pin2_comp_name,
                pin=pin2_comp_pin)),
        total_length=length)
    myoptions.update(cpw_options)
    myoptions.meander.asymmetry = asymmetry
    return RouteMeander(design, cpw_name, myoptions)

asym = 500
cpw1 = connect('cpw1', 'Q7', 'b', 'Q5', 'c', '50um', f'-{asym-1.25*offset_tm}um')
cpw3 = connect('cpw3', 'Q7', 'a', 'Q6', 'b', '50um', f'-{asym-1.25*offset_tm}um')
cpw4 = connect('cpw4', 'Q6', 'a', 'Q1', 'b', '50um', f'-{asym-1.25*offset_tm}um')
cpw5 = connect('cpw5', 'Q5', 'd', 'Q4', 'c', '50um', f'-{asym-1.25*offset_tm}um')
cpw6 = connect('cpw6', 'Q4', 'd', 'Q3', 'c', '50um', f'-{asym-1.25*offset_tm}um')
cpw7 = connect('cpw7', 'Q1', 'a', 'Q2', 'b', '50um', f'-{asym-1.25*offset_tm}um')
cpw8 = connect('cpw8', 'Q2', 'a', 'Q3', 'd', '50um', f'-{asym-1.25*offset_tm}um')





gui.rebuild()
gui.autoscale()

12:11AM 21s WARNING [check_lengths]: For path table, component=cpw4, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:11AM 21s WARNING [check_lengths]: For path table, component=cpw4, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:11AM 21s WARNING [check_lengths]: For path table, component=cpw5, key=trace has short segments that could cause issues with fillet. Values in (15-18)  are index(es) in shapely geometry.
12:11AM 21s WARNING [check_lengths]: For path table, component=cpw5, key=cut has short segments that could cause issues with fillet. Values in (15-18)  are index(es) in shapely geometry.
12:11AM 22s WARNING [check_lengths]: For path table, component=cpw4, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:11AM 22s WARNING [check_lengths]: For path table, component=cp

In [6]:

p1_q = LaunchpadWirebondCoupled(design, 'P1_Q', options = dict(pos_x='-4.25mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p2_q = LaunchpadWirebondCoupled(design, 'P2_Q', options = dict(pos_x='-1.5mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p3_q = LaunchpadWirebondCoupled(design, 'P3_Q', options = dict(pos_x='+1.0mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p4_q = LaunchpadWirebondCoupled(design, 'P4_Q', options = dict(pos_x='+2.5mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p5_q = LaunchpadWirebondCoupled(design, 'P5_Q', options = dict(pos_x='-4.25mm', pos_y='-2.5mm', orientation='90', lead_length='30um'))
p6_q = LaunchpadWirebondCoupled(design, 'P6_Q', options = dict(pos_x='-1.4mm', pos_y='-2.5mm', orientation='90', lead_length='30um'))
p7_q = LaunchpadWirebondCoupled(design, 'P7_Q', options = dict(pos_x='+2.5mm', pos_y='-2.5mm', orientation='90', lead_length='30um'))

gui.rebuild()
gui.autoscale()

fillet = '99.99um'

def connect(cpw_name: str, pin1_comp_name: str, pin1_comp_pin: str, pin2_comp_name: str, pin2_comp_pin: str,
            length: str, asymmetry='0 um'):
    myoptions = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component=pin1_comp_name, pin=pin1_comp_pin),
            end_pin=Dict(component=pin2_comp_name, pin=pin2_comp_pin)),
        total_length=length)
    myoptions.update(cpw_options)
    myoptions.meander.asymmetry = asymmetry
    return RouteMeander(design, cpw_name, myoptions)

cpw_options = Dict(
    lead=Dict(
        start_straight='430um',
        end_straight='0um'),
    fillet=fillet
)
asym = 150
ol1 = connect('ol1', 'Q7', 'c', 'P1_Q', 'tie', '8.6 mm', f'+{asym}um')
ol3 = connect('ol3', 'Q1', 'd', 'P3_Q', 'tie', '8.6 mm', f'+{asym}um')

cpw_options = Dict(
    lead=Dict(
        start_straight='535um',
        end_straight='0um'),
    fillet=fillet
)
asym = 200
ol2 = connect('ol2', 'Q6', 'd', 'P2_Q', 'tie', '8.6 mm', f'+{asym}um')
ol4 = connect('ol4', 'Q2', 'd', 'P4_Q', 'tie', '8.6 mm', f'+{asym}um')

cpw_options = Dict(
    lead=Dict(
        start_straight='480um',
        end_straight='0um'),
    fillet=fillet
)
asym = 180
ol5 = connect('ol5', 'Q5', 'b', 'P5_Q', 'tie', '8.6 mm', f'+{asym}um')
ol6 = connect('ol6', 'Q4', 'b', 'P6_Q', 'tie', '8.6 mm', f'+{asym}um')
ol7 = connect('ol7', 'Q3', 'a', 'P7_Q', 'tie', '8.6 mm', f'+{asym}um')

gui.rebuild()
gui.autoscale()
#gui.highlight_components(['Q1','Q2','Q3','Q4','Q5','Q6','Q7'])

12:11AM 23s WARNING [check_lengths]: For path table, component=cpw4, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:11AM 23s WARNING [check_lengths]: For path table, component=cpw4, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:11AM 23s WARNING [check_lengths]: For path table, component=cpw5, key=trace has short segments that could cause issues with fillet. Values in (15-18)  are index(es) in shapely geometry.
12:11AM 23s WARNING [check_lengths]: For path table, component=cpw5, key=cut has short segments that could cause issues with fillet. Values in (15-18)  are index(es) in shapely geometry.
12:11AM 25s WARNING [check_lengths]: For path table, component=cpw4, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:11AM 25s WARNING [check_lengths]: For path table, component=cp